Process the raw ElVis CSV files


In [48]:
import os
import glob
import pandas as pd
import numpy as np
import sklearn.utils

np.random.seed(10)

os.chdir("./")

In [49]:
#Get the raw data for a problem and combine the archives into a single run
def getData(problem):
    extension = 'ElVis.csv'
    all_filenames = [i for i in glob.glob('*'+problem+'*{}'.format(extension))]
    print(problem)
      
    #combine all files in the list
    combined_f1 = pd.concat([pd.read_csv(f, skiprows=3) for f in all_filenames ])
    
    combined_f1.key = combined_f1.key.astype(str)
    #sort 
    combined_f1 = combined_f1.sort_values(["key","dist"]) 
    
    # dropping duplicate values 
    combined_f1 = combined_f1.drop_duplicates(keep=False) 
    combined_f1 = combined_f1.dropna(axis=1, thresh=1)

    #get list of keys
    keys = combined_f1.key
    keys.drop_duplicates( keep='first', inplace=True)

    combined_f1 = combined_f1.reset_index(drop=True)

    final_f1 = pd.DataFrame()
    #Now ensure only 1 entry per key
    for k in keys:
        extract = combined_f1.loc[combined_f1["key"] == k]
        extract = extract.sort_values("dist")
        final_f1 = final_f1.append(extract.head(1))

    #export to csv
    ! del "all"+problem+".csv"
    final_f1.to_csv( "all"+problem+".csv", index=False, encoding='utf-8-sig')
    print("Combined" + str(len(final_f1)))
    return final_f1


Now get the combined elites for each problem

In [50]:
f1 = getData("f1")
f2 = getData("f2")
f6 = getData("f6")
f8 = getData("f8")
fcity = getData("fc")

f1
/bin/sh: del: command not found
Combined1137
f2
/bin/sh: del: command not found
Combined113
f6
/bin/sh: del: command not found
Combined1523
f8
/bin/sh: del: command not found
Combined573
fc
/bin/sh: del: command not found
Combined514


In [51]:
#def classifier(tab, col):
#    tab[col] = tab[col].astype(str)
#    tab[col] = tab[col].str.replace('1','very_low')
#    tab[col] = tab[col].str.replace('2','low')
#    tab[col] = tab[col].str.replace('3','medium')
#    tab[col] = tab[col].str.replace('4','high')
#    tab[col] = tab[col].str.replace('5','very_high')
#    return tab

Create copies of the file for each characteristic

In [53]:
def balance(table, col):
    low = (table[table[col]==1])
    high = (table[table[col]>1])
    print ("Unbalanced " + str(len(low) )+ ":"+str(len(high)))
    if (len(low.index) < len(high.index)):
                
        #randomise high
        # Shuffle Pandas data frame
        high = sklearn.utils.shuffle(high)
        high = high.head(len(low.index))
    
    else:
        low = sklearn.utils.shuffle(low)
        low = low.head(len(high.index))
    
    high = high.append(low)
    print("balanced " + str(len(high)))
    return high
    

In [ ]:
f1

In [55]:
#[['byMD','MDs in use','WalkDels','CycleDels','EvanDels','Time','fixed cost','running cost''emissions']]


em = f1.copy()
em = em.append(f2)
em = em.append(f6)
em = em.append(f8)
em = em.append(fcity)
em = em [['byMD','MDs in use','WalkDels','CycleDels','EvanDels','emissions']]
em = balance(em,'emissions')
em.to_csv(r'emOut.csv', index = False)

t = f1.copy()
t = t.append(f2)
t = t.append(f6)
t = t.append(f8)
t = t.append(fcity)
t = t [['byMD','MDs in use','WalkDels','CycleDels','EvanDels','Time']]
t = balance(t,'Time')
t.to_csv(r'tOut.csv', index = False)

fc = f1.copy()
fc = fc.append(f2)
fc = fc.append(f6)
fc = fc.append(f8)
fc = fc.append(fcity)
fc = fc [['byMD','MDs in use','WalkDels','CycleDels','EvanDels','fixed cost']]
fc = balance(fc,'fixed cost')
fc.to_csv(r'fcOut.csv', index = False)

rc = f1.copy()
rc = rc.append(f2)
rc = rc.append(f6)
rc = rc.append(f8)
rc = rc.append(fcity)
rc = rc [['byMD','MDs in use','WalkDels','CycleDels','EvanDels','running cost']]
#There are no rc examples that catogorise as 1....
#So we will rename all of the 2s as 1s
rc['running cost'].replace(2, 1, inplace=True)
rc = balance(rc,'running cost')
rc.to_csv(r'rcOut.csv', index = False)


Unbalanced 1609:2251
balanced 3218
Unbalanced 535:3325
balanced 1070
Unbalanced 166:3694
balanced 332
Unbalanced 35:3825
balanced 70


In [27]:
t

,byMD,MDs in use,WalkDels,CycleDels,EvanDels,Time
1901,5,1,2,2,1,4
1324,2,1,1,2,1,5
1367,2,1,1,1,1,4
1362,3,1,1,2,1,4
769,3,1,1,1,2,4
...,...,...,...,...,...,...
634,4,2,1,3,1,1
635,4,2,1,3,1,1
636,4,2,1,3,2,1
667,4,3,1,1,3,1


Place stats here...


In [10]:
f2

,key,dist,emissions,byMD,MDs in use,Time,WalkDels,CycleDels,EvanDels,fixed cost,...,MDs in use.1,Van Dist,Van Dels,MD dels,Dist walk,Dist cycle,Dist eVan,Dels walk,Dels cycle,Dels eVan
0,1:4:1:2:1:1:4:2:4,0.377007,1,4,1,2,1,1,4,2,...,2,4.472052,3,22,0.000000,3.356240,10.780986,0,4,18
1,1:4:1:3:1:1:3:4:5,0.901599,1,4,1,3,1,1,3,4,...,4,6.130288,10,64,0.000000,37.117088,65.623587,0,16,48
2,1:4:1:3:1:1:4:2:5,0.579843,1,4,1,3,1,1,4,2,...,3,12.666451,9,65,0.000000,19.495532,43.170038,0,6,59
3,1:4:1:3:1:2:3:3:5,0.683656,1,4,1,3,1,2,3,3,...,4,6.943545,9,65,0.000000,36.722122,42.487591,0,26,39
5,1:4:1:3:1:2:3:5:5,0.957860,1,4,1,3,1,2,3,5,...,4,7.468818,10,64,0.000000,35.901964,71.364618,0,20,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,3:4:1:3:1:1:3:2:5,0.544998,3,4,1,3,1,1,3,2,...,4,10.444541,8,17,1.172542,6.121735,6.883970,2,3,12
150,3:4:1:3:1:2:3:2:5,0.804642,3,4,1,3,1,2,3,2,...,4,26.932033,15,59,0.000000,32.364922,39.467428,0,18,41
151,3:4:2:5:1:1:3:5:5,1.000000,3,4,2,5,1,1,3,5,...,8,26.264103,12,62,31.822963,37.181031,70.241550,12,14,36
152,3:4:2:5:2:2:2:5:5,1.000000,3,4,2,5,2,2,2,5,...,10,30.498172,10,64,67.873479,57.956195,88.154945,18,18,28


In [11]:
def stats(table, col):
    print(len(table))
    print("High " + str(len(table[table[col]==1])))
    print("Low " + str(len(table[table[col]>1])))


print("Time")
stats(t,"Time")
print("Emissions")
stats(em,"emissions")
print("Running Cost")
stats(rc,"running cost")
print("Fixed Cost")
stats(fc,"fixed cost")

Time
1070
High 535
Low 535
Emissions
3218
High 1609
Low 1609
Running Cost
70
High 35
Low 35
Fixed Cost
332
High 166
Low 166


In [12]:
def insert_row(row_number, df, row_value): 
    # Starting value of upper half 
    start_upper = 0
   
    # End value of upper half 
    end_upper = row_number 
   
    # Start value of lower half 
    start_lower = row_number 
   
    # End value of lower half 
    end_lower = df.shape[0] 
   
    # Create a list of upper_half index 
    upper_half = [*range(start_upper, end_upper, 1)] 
   
    # Create a list of lower_half index 
    lower_half = [*range(start_lower, end_lower, 1)] 
   
    # Increment the value of lower half by 1 
    lower_half = [x.__add__(1) for x in lower_half] 
   
    # Combine the two lists 
    index_ = upper_half + lower_half 
   
    # Update the index of the dataframe 
    df.index = index_ 
   
    # Insert a row at the end 
    df.loc[row_number] = row_value 
    
    # Sort the index labels 
    df = df.sort_index() 
   
    # return the dataframe 
    return df 
   

In [13]:
def normCol(old,col):
    tab = old.copy()
    tab[col] = tab[col].astype(str)
    tab[col] = tab[col].str.replace('1.0','very_low')
    tab[col] = tab[col].str.replace('2.0','low')
    tab[col] = tab[col].str.replace('3.0','medium')
    tab[col] = tab[col].str.replace('4.0','high')
    tab[col] = tab[col].str.replace('5.0','very_high')
    return tab

In [14]:





em_o_tab  = em.copy()
em_o_tab = normCol(em_o_tab,'emissions')
    #add additional headers ...
    
row1 = ['c','c','c','c','c','d',]
row2 = ['','','','','','class']
em_o_tab = insert_row(0,em_o_tab, row1)
em_o_tab = insert_row(1,em_o_tab, row2)
!del em-o-all.csv
em_o_tab.to_csv( "em-o-all.csv", index=False, encoding='utf-8-sig')
   
t_o_tab  = t.copy()
#add additional headers ...
t_o_tab = insert_row(0,t_o_tab, row1)
t_o_tab = insert_row(1,t_o_tab, row2)
!del t-o-all.csv
t_o_tab.to_csv( "t-o-all.csv", index=False, encoding='utf-8-sig')

fc_o_tab  = fc.copy()
#add additional headers ...
fc_o_tab = insert_row(0,fc_o_tab, row1)
fc_o_tab = insert_row(1,fc_o_tab, row2)
!!del fc-o-all.csv
fc_o_tab.to_csv( "fc-o-all.csv", index=False, encoding='utf-8-sig')

rc_o_tab  = rc.copy()

    
#add additional headers ...

rc_o_tab = insert_row(0,rc_o_tab, row1)
rc_o_tab = insert_row(1,rc_o_tab, row2)
!del rc-o-all.csv
rc_o_tab.to_csv( "rc-o-all.csv", index=False, encoding='utf-8-sig')




/bin/sh: del: command not found
/bin/sh: del: command not found
/bin/sh: del: command not found


Let's use Orange to find some CN2 rules!

Install Orange3 using pip. Check formatting of data file See https://docs.biolab.si//3/data-mining-library/reference/classification.html#module-Orange.classification.rules Need to reformat the ElVis file for use with Orange, see https://orange-visual-programming.readthedocs.io/loading-your-data/index.html#three-row-header-format

In [15]:
#Imports
import Orange
import re
fileSizes = {}

rulesDB = pd.DataFrame()#the master list of all rules found, ever
pd.options.display.max_colwidth = 100
pd.set_option('display.max_rows', 1000)


In [16]:
#Filter rules 
#Remove any rules with constants (True or False)
#Only allow rules that result in a 1 for the final value
def filterR(rules):

    selected = []
    for r in rules:
        
        
        if (re.search("THEN.*1", str(r))):#or (re.search("THEN.*2", str(r))):
            if not 'TRUE' in str(r): # Remove rules with constants
                if not 'FALSE' in str(r): # Remove rules with constants
                    selected.append(r)
    return selected

In [17]:
# This function returns a list of rules. 
#We use the CN2 learner
#DataTable is the output from convElVis()  and covered_examples is the minimim number of examples that a potential rule
# must satisfy

def learnRules(dataTable,covered_examples):
    #Learn the rules from the Dataset
    data = Orange.data.Table(dataTable)
    learner = Orange.classification.CN2UnorderedLearner()

    # consider up to 10 solution streams at one time
    learner.rule_finder.search_algorithm.beam_width = 10

    # continuous value space is constrained to reduce computation time
    learner.rule_finder.search_strategy.constrain_continuous = True

    # found rules must cover at least 15 examples
    learner.rule_finder.general_validator.min_covered_examples = covered_examples

    # found rules may combine at most 2 selectors (conditions)
    learner.rule_finder.general_validator.max_rule_length = 5

    classifier = learner(data)

    return classifier.rule_list

In [18]:
#def extractRules(orangeData, expName):
#    print("Extracting rules from " +orangeData)
#    print("Coverage requirement ")
#    ruleList = pd.DataFrame()
#    for coverThreshold in coverage:
#        print(coverThreshold)
#        rules = learnRules(orangeData,coverThreshold)
#        
#        rules = filterR(rules)
#        #add rules to dataframe
#        for rule in rules:               
#            row = pd.Series([str(rule),coverThreshold],index=['rule','coverage'])
#            ruleList = ruleList.append(row,ignore_index=True)
              
#    return ruleList #, selected

def extractRules(orangeData, expName):
    print("Extracting rules from " +orangeData)
    print("Coverage requirement ")
    ruleList = pd.DataFrame()
    rules = learnRules(orangeData,10)
        
    rules = filterR(rules)
    for rule in rules:
        row = pd.Series([rule,rule.quality,rule.complexity,rule.length],index=['rule','quality','complexity','length'])
        ruleList = ruleList.append(row,ignore_index=True)
        #Quality is based on Laplace Accuracy
              
    return ruleList #, selected

In [19]:
def makeReadable(rulesDB):
    print("OK")
    #change outputs in rule DB
    rulesDB = rulesDB.replace(to_replace ='1.0',  value =' = Very Low', regex=True) 
    rulesDB = rulesDB.replace(to_replace ='2.0',  value =' = Low', regex=True) 
    rulesDB = rulesDB.replace(to_replace ='3.0',  value =' = Medium', regex=True) 
    rulesDB = rulesDB.replace(to_replace ='4.0',  value =' = High', regex=True) 
    rulesDB = rulesDB.replace(to_replace ='5.0',  value =' = Very High', regex=True) 
    rulesDB = rulesDB.replace(to_replace ='=1',  value =' = Very Low', regex=True) 
    rulesDB = rulesDB.replace(to_replace ='=2',  value =' = Low', regex=True) 
    rulesDB = rulesDB.replace(to_replace ='=3',  value =' = Medium', regex=True) 
    rulesDB = rulesDB.replace(to_replace ='=4',  value =' = High', regex=True) 
    rulesDB = rulesDB.replace(to_replace ='=5',  value =' = Very High', regex=True) 
    return rulesDB

In [20]:
#Check all problems, collectively

time_o_rules = pd.DataFrame()
experiment = "Time"
time_o_rules =  extractRules("t-o-all.csv",experiment)
time_o_rules = makeReadable(time_o_rules)

em_o_rules = pd.DataFrame()
experiment = "emissions"
em_o_rules =  extractRules("em-o-all.csv",experiment)
em_o_rules = makeReadable(em_o_rules)

fc_o_rules = pd.DataFrame()
experiment = "fixed cost"
fc_o_rules =  extractRules("fc-o-all.csv",experiment)
fc_o_rules = makeReadable(fc_o_rules)

rc_o_rules = pd.DataFrame()
experiment = "running cost"
rc_o_rules =  extractRules("rc-o-all.csv",experiment)
rc_o_rules = makeReadable(rc_o_rules)



Extracting rules from t-o-all.csv
Coverage requirement 
Extracting rules from em-o-all.csv
Coverage requirement 
Extracting rules from fc-o-all.csv
Coverage requirement 
Extracting rules from rc-o-all.csv
Coverage requirement 


In [71]:
em_o_rules.to_csv( "em_o_rules.csv", index=False, encoding='utf-8-sig')
time_o_rules.to_csv( "t_o_rules.csv", index=False, encoding='utf-8-sig')
rc_o_rules.to_csv( "rc_o_rules.csv", index=False, encoding='utf-8-sig')
fc_o_rules.to_csv( "fc_o_rules.csv", index=False, encoding='utf-8-sig')


------------------------------------------

Now use FlexGP

------------------------------------------

In [157]:
#Prepare data for flexGP
flex_all = f1.copy()
flex_all = flex_all.append(f2)
flex_all = flex_all.append(f6)
flex_all = flex_all.append(f8)
flex_all = flex_all.append(fcity)

flex_all = flex_all.drop(["key","dist","Van Dist","Van Dels","MD dels","Dist walk","Dist cycle","Dels","Emissions",
                       "Actual running cost","actual emissions","Actual fixed cost","Fitness","Timesspan","Fixed Cost",
                       "Running cost","Actual Time","Actual WalkDels","Dist eVan","Dels walk","Dels cycle",
                       "actual byMd","Actual MDs in use","Actual CycleDels","Actual EvanDels","MDs in use.1",
                        "Dels eVan"],axis=1)

flex_all['byMD'] = flex_all['byMD'].astype(float)
flex_all['MDs in use'] = flex_all['MDs in use'].astype(float)
flex_all['WalkDels'] = flex_all['WalkDels'].astype(float)
flex_all['CycleDels'] = flex_all['CycleDels'].astype(float)
flex_all['EvanDels'] = flex_all['EvanDels'].astype(float)


flex_all

,emissions,byMD,MDs in use,Time,WalkDels,CycleDels,EvanDels,fixed cost,running cost
0,1,3.0,1.0,3,1.0,3.0,1.0,1,3
1,1,4.0,1.0,1,1.0,1.0,4.0,2,3
2,1,4.0,1.0,1,1.0,1.0,4.0,3,4
3,1,4.0,1.0,1,1.0,1.0,4.0,4,4
5,1,4.0,1.0,1,1.0,2.0,2.0,2,3
...,...,...,...,...,...,...,...,...,...
719,3,2.0,2.0,5,1.0,1.0,2.0,3,4
720,3,2.0,2.0,5,1.0,1.0,2.0,4,4
721,3,2.0,2.0,5,1.0,1.0,2.0,5,4
722,3,2.0,2.0,5,1.0,2.0,1.0,2,4


In [158]:
#Predict emissions using flexGP
flex_all_em = flex_all.copy()
flex_all_em = flex_all_em.drop(["Time","fixed cost","running cost"],axis=1)
# change any emissions > 1 to 0
flex_all_em.loc[(flex_all_em.emissions > 1),'emissions']='0'
flex_all_em = flex_all_em[['byMD','MDs in use','WalkDels','CycleDels','EvanDels','emissions']]

!rm flex_all_em.csv
flex_all_em.to_csv( "flex_all_em.csv", index=False,header=False)
!java -jar ruletree.jar -train flex_all_em.csv -minutes 5 
!java -jar ruletree.jar -test flex_all_em.csv >> rules.txt
!mkdir gp_em
!mv *.txt ./gp_em/
flex_all_em


Running evogpj with seed: 1603653627354
[X1, X2, X3, X4, X5]
Computing conditions with False Negative cost: 0.5 and False Positive cost: 0.5
[C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11]
GENERATE TREE!
Generation successful!
{fitness.RT_FP_Fitness.Java=0.6423811639271435, fitness.RT_FN_Fitness.Java=0.08576755748912368, fitness.SubtreeComplexity=1.0}
ELAPSED TIME: 2
Generation 0
Best individual for generation 0:
{fitness.RT_FP_Fitness.Java=0.6423811639271435, fitness.RT_FN_Fitness.Java=0.08576755748912368, fitness.SubtreeComplexity=1.0}
ELAPSED TIME: 4
Generation 1
Best individual for generation 1:
{fitness.RT_FP_Fitness.Java=0.6423811639271435, fitness.RT_FN_Fitness.Java=0.08576755748912368, fitness.SubtreeComplexity=1.0}
ELAPSED TIME: 7
Generation 2
Best individual for generation 2:
{fitness.RT_FP_Fitness.Java=0.6423811639271435, fitness.RT_FN_Fitness.Java=0.08576755748912368, fitness.SubtreeComplexity=1.0}
ELAPSED TIME: 9
Generation 3
Best individual for generation 3:
{fitness.RT_FP

Best individual for generation 41:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=29.0}
ELAPSED TIME: 41
Generation 42
Best individual for generation 42:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=29.0}
ELAPSED TIME: 43
Generation 43
Best individual for generation 43:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=29.0}
ELAPSED TIME: 44
Generation 44
Best individual for generation 44:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=29.0}
ELAPSED TIME: 45
Generation 45
Best individual for generation 45:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=29.0}
ELAPSED TIME: 46
Generation 46
Best individual for generation

Best individual for generation 84:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 93
Generation 85
Best individual for generation 85:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 95
Generation 86
Best individual for generation 86:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 96
Generation 87
Best individual for generation 87:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 98
Generation 88
Best individual for generation 88:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 99
Generation 89
Best individual for generation

Best individual for generation 126:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 165
Generation 127
Best individual for generation 127:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 166
Generation 128
Best individual for generation 128:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 168
Generation 129
Best individual for generation 129:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 169
Generation 130
Best individual for generation 130:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 171
Generation 131
Best individual

Best individual for generation 168:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 228
Generation 169
Best individual for generation 169:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 230
Generation 170
Best individual for generation 170:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 232
Generation 171
Best individual for generation 171:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 233
Generation 172
Best individual for generation 172:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 235
Generation 173
Best individual

Best individual for generation 210:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 293
Generation 211
Best individual for generation 211:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 294
Generation 212
Best individual for generation 212:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 295
Generation 213
Best individual for generation 213:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 296
Generation 214
Best individual for generation 214:
{fitness.RT_FP_Fitness.Java=0.5979564637938694, fitness.RT_FN_Fitness.Java=0.12865133623368552, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 297
Generation 215
Best individual

,byMD,MDs in use,WalkDels,CycleDels,EvanDels,emissions
0,3.0,1.0,1.0,3.0,1.0,1
1,4.0,1.0,1.0,1.0,4.0,1
2,4.0,1.0,1.0,1.0,4.0,1
3,4.0,1.0,1.0,1.0,4.0,1
5,4.0,1.0,1.0,2.0,2.0,1
...,...,...,...,...,...,...
719,2.0,2.0,1.0,1.0,2.0,0
720,2.0,2.0,1.0,1.0,2.0,0
721,2.0,2.0,1.0,1.0,2.0,0
722,2.0,2.0,1.0,2.0,1.0,0


In [159]:
#Predict time using flexGP
flex_all_t = flex_all.copy()
flex_all_t = flex_all_t.drop(["emissions","fixed cost","running cost"],axis=1)
# change any time > 1 to 0
flex_all_t.loc[(flex_all_t.Time > 1),'Time']='0'
flex_all_t = flex_all_t[['byMD','MDs in use','WalkDels','CycleDels','EvanDels','Time']]
!rm flex_all_t.csv
flex_all_t.to_csv( "flex_all_t.csv", index=False,header=False)
!java -jar ruletree.jar -train flex_all_t.csv -minutes 5 
!java -jar ruletree.jar -test flex_all_t.csv >> rules.txt
!mkdir gp_t
!mv *.txt ./gp_t/

Running evogpj with seed: 1603653931064
[X1, X2, X3, X4, X5]
Computing conditions with False Negative cost: 0.5 and False Positive cost: 0.5
[C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11, C12]
GENERATE TREE!
Generation successful!
{fitness.RT_FP_Fitness.Java=0.6664661654135339, fitness.RT_FN_Fitness.Java=0.03177570093457944, fitness.SubtreeComplexity=159.0}
ELAPSED TIME: 2
Generation 0
Best individual for generation 0:
{fitness.RT_FP_Fitness.Java=0.6664661654135339, fitness.RT_FN_Fitness.Java=0.03177570093457944, fitness.SubtreeComplexity=37.0}
ELAPSED TIME: 6
Generation 1
Best individual for generation 1:
{fitness.RT_FP_Fitness.Java=0.6664661654135339, fitness.RT_FN_Fitness.Java=0.03177570093457944, fitness.SubtreeComplexity=8.0}
ELAPSED TIME: 8
Generation 2
Best individual for generation 2:
{fitness.RT_FP_Fitness.Java=0.6664661654135339, fitness.RT_FN_Fitness.Java=0.03177570093457944, fitness.SubtreeComplexity=8.0}
ELAPSED TIME: 10
Generation 3
Best individual for generation 3:
{fitn

Best individual for generation 41:
{fitness.RT_FP_Fitness.Java=0.6499248120300752, fitness.RT_FN_Fitness.Java=0.044859813084112146, fitness.SubtreeComplexity=29.0}
ELAPSED TIME: 88
Generation 42
Best individual for generation 42:
{fitness.RT_FP_Fitness.Java=0.6499248120300752, fitness.RT_FN_Fitness.Java=0.044859813084112146, fitness.SubtreeComplexity=29.0}
ELAPSED TIME: 89
Generation 43
Best individual for generation 43:
{fitness.RT_FP_Fitness.Java=0.6499248120300752, fitness.RT_FN_Fitness.Java=0.044859813084112146, fitness.SubtreeComplexity=29.0}
ELAPSED TIME: 91
Generation 44
Best individual for generation 44:
{fitness.RT_FP_Fitness.Java=0.6499248120300752, fitness.RT_FN_Fitness.Java=0.044859813084112146, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 92
Generation 45
Best individual for generation 45:
{fitness.RT_FP_Fitness.Java=0.6499248120300752, fitness.RT_FN_Fitness.Java=0.044859813084112146, fitness.SubtreeComplexity=27.0}
ELAPSED TIME: 93
Generation 46
Best individual for gener

In [160]:
#Predict fixed cost using flexGP
flex_all_fc = flex_all.copy()
flex_all_fc = flex_all_fc.drop(["emissions","Time","running cost"],axis=1)
# change any fc > 1 to 0
flex_all_fc.rename(columns = {'fixed cost':'fixed_cost'}, inplace = True)
flex_all_fc.loc[(flex_all_fc.fixed_cost > 1),'fixed_cost']='0'
flex_all_fc = flex_all_fc[['byMD','MDs in use','WalkDels','CycleDels','EvanDels','fixed_cost']]
!rm flex_all_fc.csv
flex_all_fc.to_csv( "flex_all_fc.csv", index=False,header=False)
!java -jar ruletree.jar -train flex_all_fc.csv -minutes 5

!java -jar ruletree.jar -test flex_all_fc.csv >> rules.txt
!mkdir gp_fc
!mv *.txt ./gp_fc/

Running evogpj with seed: 1603654543670
[X1, X2, X3, X4, X5]
Computing conditions with False Negative cost: 0.5 and False Positive cost: 0.5
[C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11, C12, C13, C14, C15, C16, C17, C18, C19, C20, C21, C22, C23, C24, C25, C26, C27, C28, C29, C30, C31, C32, C33, C34, C35, C36, C37, C38, C39, C40, C41, C42, C43, C44, C45, C46, C47, C48, C49, C50, C51, C52, C53, C54, C55, C56, C57, C58, C59, C60, C61, C62, C63, C64, C65, C66, C67, C68, C69, C70, C71, C72, C73, C74, C75, C76, C77, C78, C79, C80, C81, C82, C83, C84, C85, C86, C87, C88, C89, C90, C91, C92, C93, C94, C95, C96, C97, C98, C99, C100, C101, C102, C103, C104, C105, C106, C107, C108, C109]
GENERATE TREE!
Generation successful!
{fitness.RT_FP_Fitness.Java=0.28749323226854356, fitness.RT_FN_Fitness.Java=0.0, fitness.SubtreeComplexity=1.0}
ELAPSED TIME: 6
Generation 0
Best individual for generation 0:
{fitness.RT_FP_Fitness.Java=0.28749323226854356, fitness.RT_FN_Fitness.Java=0.0, fitness.SubtreeCom

Best individual for generation 38:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=51.0}
ELAPSED TIME: 84
Generation 39
Best individual for generation 39:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=51.0}
ELAPSED TIME: 85
Generation 40
Best individual for generation 40:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 86
Generation 41
Best individual for generation 41:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 88
Generation 42
Best individual for generation 42:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 89
Generation 43
Best individual for 

Best individual for generation 80:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 145
Generation 81
Best individual for generation 81:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 146
Generation 82
Best individual for generation 82:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 149
Generation 83
Best individual for generation 83:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 151
Generation 84
Best individual for generation 84:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 154
Generation 85
Best individual

Best individual for generation 122:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 218
Generation 123
Best individual for generation 123:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 219
Generation 124
Best individual for generation 124:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 220
Generation 125
Best individual for generation 125:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 221
Generation 126
Best individual for generation 126:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 223
Generation 127
Best 

{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 279
Generation 164
Best individual for generation 164:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 281
Generation 165
Best individual for generation 165:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 282
Generation 166
Best individual for generation 166:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 284
Generation 167
Best individual for generation 167:
{fitness.RT_FP_Fitness.Java=0.19328641039523553, fitness.RT_FN_Fitness.Java=0.018072289156626505, fitness.SubtreeComplexity=43.0}
ELAPSED TIME: 285
Generation 168
Best individual for generation 168:
{fitn

In [161]:
#Predict running cost using flexGP
flex_all_rc = flex_all.copy()
flex_all_rc = flex_all_rc.drop(["emissions","Time","fixed cost"],axis=1)
# change any rc > 1 to 0
flex_all_rc.rename(columns = {'running cost':'running_cost'}, inplace = True)
#Running cost has to be > 2
flex_all_rc.loc[(flex_all_rc.running_cost > 2),'running_cost']='0'
flex_all_rc.loc[(flex_all_rc.running_cost == 2),'running_cost']='1'

flex_all_rc = flex_all_rc[['byMD','MDs in use','WalkDels','CycleDels','EvanDels','running_cost']]
!rm flex_all_rc.csv
flex_all_rc.to_csv( "flex_all_rc.csv", index=False,header=False)
!java -jar ruletree.jar -train flex_all_rc.csv -minutes 5
!java -jar ruletree.jar -test flex_all_rc.csv >> rules.txt
!mkdir gp_rc
!mv *.txt ./gp_rc/

Running evogpj with seed: 1603654848513
[X1, X2, X3, X4, X5]
Computing conditions with False Negative cost: 0.5 and False Positive cost: 0.5
[C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11, C12, C13, C14, C15, C16, C17, C18, C19, C20, C21, C22, C23, C24, C25, C26, C27, C28, C29, C30, C31, C32, C33, C34, C35, C36, C37, C38, C39, C40, C41, C42, C43, C44, C45, C46, C47, C48, C49, C50, C51, C52, C53, C54, C55, C56, C57, C58, C59, C60, C61, C62, C63, C64, C65, C66, C67, C68, C69, C70, C71, C72, C73, C74, C75, C76, C77, C78, C79, C80, C81, C82, C83, C84, C85, C86, C87, C88, C89, C90, C91, C92, C93, C94, C95, C96, C97, C98, C99, C100, C101, C102, C103, C104, C105, C106, C107, C108]
GENERATE TREE!
Generation successful!
{fitness.RT_FP_Fitness.Java=0.23973856209150327, fitness.RT_FN_Fitness.Java=0.2571428571428571, fitness.SubtreeComplexity=1.0}
ELAPSED TIME: 3
Generation 0
Best individual for generation 0:
{fitness.RT_FP_Fitness.Java=0.23973856209150327, fitness.RT_FN_Fitness.Java=0.257142857142

Best individual for generation 41:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 82
Generation 42
Best individual for generation 42:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 83
Generation 43
Best individual for generation 43:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 85
Generation 44
Best individual for generation 44:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 86
Generation 45
Best individual for generation 45:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 88
Generation 46
Best individual for generation 46:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness

Best individual for generation 87:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 142
Generation 88
Best individual for generation 88:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 143
Generation 89
Best individual for generation 89:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 144
Generation 90
Best individual for generation 90:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 145
Generation 91
Best individual for generation 91:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 147
Generation 92
Best individual for generation 92:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fi

Best individual for generation 133:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 202
Generation 134
Best individual for generation 134:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 204
Generation 135
Best individual for generation 135:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 206
Generation 136
Best individual for generation 136:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 207
Generation 137
Best individual for generation 137:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 209
Generation 138
Best individual for generation 138:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitne

Best individual for generation 179:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 262
Generation 180
Best individual for generation 180:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 264
Generation 181
Best individual for generation 181:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 265
Generation 182
Best individual for generation 182:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 266
Generation 183
Best individual for generation 183:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitness.RT_FN_Fitness.Java=0.2, fitness.SubtreeComplexity=25.0}
ELAPSED TIME: 267
Generation 184
Best individual for generation 184:
{fitness.RT_FP_Fitness.Java=0.21830065359477124, fitne

Extract GP rules

In [41]:
def isfloat(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def makeReadable(condition):
    
    toks = condition.split(' ')
    
    line= ''
    for token in toks:
        token = token.strip()
        if ';' in token:
            token = '-'
        
        
        if isfloat(token):
            f = float(token)
            token = str(round(f,2))
            
        line = line + token
        
        line = line + ' '
    return line

def getConditions(file):
    with open(file) as f:
        conditions = pd.DataFrame(columns=['id', 'condition'])
        for line in f:
            data = line.strip().split(':')
            
            condition = makeReadable(data[1].strip())
            conditions = conditions.append({'id': data[0].strip(), 'condition': condition}, ignore_index=True)

    #Replace X1 etc with actual names

    conditions['condition'].replace('X1','byMD', regex=True, inplace=True)
    conditions['condition'].replace('X2','MDSinUse', regex=True, inplace=True)
    conditions['condition'].replace('X3','WalkDels', regex=True, inplace=True)
    conditions['condition'].replace('X4','CycleDels', regex=True, inplace=True)
    conditions['condition'].replace('X5','EVanDels', regex=True, inplace=True)

    
    return conditions


    
def subConditions(rule, conditions):
    for index, row in conditions.iterrows():
        id = row['id']
        condition = row['condition']
        rule = rule.replace(str(id+' '),condition)
        rule = rule.replace(str(id+")"),condition+")")
        if (rule.strip() == id.strip()):
            rule = condition
    return rule

def getRules(file,conditions):
    with open(file) as file_in:
        data = []
        for line in file_in:
            data.append(line)
        
    ruleList = pd.DataFrame()
    found = False
    for line in data:
        if "TESTING PARETO FRONT:" in line:
            found = True
        
        if (found):
            if "RULE" in line:
                rule = line.strip().split(':')[1]
            if "ACCURACY" in line:
                accuracy = line.strip().split(':')[1]
            if "PRECISION" in line:
                precision = line.strip().split(':')[1]
            if "RECALL" in line:
                recall = line.strip().split(':')[1]
            if "F-SCORE" in line:
                f_score = line.strip().split(':')[1]
            if "POSITIVE" in line:
                f_positive = line.strip().split(':')[1]
            if "NEGATIVE" in line:
                f_negative = line.strip().split(':')[1]
                #Process it all
                rule = subConditions(rule, conditions)
                row = pd.Series([rule,accuracy,precision,recall,f_score,f_positive,f_negative],index=['rule','accuracy','precison','recall','f_score','false_pos','false_neg'])
                ruleList = ruleList.append(row,ignore_index=True) 
    return ruleList

In [72]:

rc_flex_rules = getRules("./gp_rc/rules.txt",getConditions("./gp_rc/conditions.txt"))
fc_flex_rules = getRules("./gp_fc/rules.txt",getConditions("./gp_fc/conditions.txt"))
em_flex_rules = getRules("./gp_em/rules.txt",getConditions("./gp_em/conditions.txt"))
t_flex_rules = getRules("./gp_t/rules.txt",getConditions("./gp_t/conditions.txt"))

em_flex_rules.to_csv("em_flex_rules.csv", index=False, encoding='utf-8-sig')
t_flex_rules.to_csv("t_flex_rules.csv", index=False, encoding='utf-8-sig')
rc_flex_rules.to_csv("rc_flex_rules.csv", index=False, encoding='utf-8-sig')
fc_flex_rules.to_csv("fc_flex_rules.csv", index=False, encoding='utf-8-sig')



# Compare our two methods using the independant checker

In [58]:
!java Checker CN2 emOut.csv em_o_rules.csv > em_o_checked.csv
!java Checker GP emOut.csv em_flex_rules.csv > em_flex_checked.csv
em_o_rules_checked = pd.read_csv("em_o_checked.csv")
em_flex_rules_checked = pd.read_csv("em_flex_checked.csv")


!java Checker CN2 tOut.csv t_o_rules.csv > t_o_checked.csv
!java Checker GP tOut.csv t_flex_rules.csv > t_flex_checked.csv
t_o_rules_checked = pd.read_csv("t_o_checked.csv")
t_flex_rules_checked = pd.read_csv("t_flex_checked.csv")

!java Checker CN2 rcOut.csv rc_o_rules.csv > rc_o_checked.csv
!java Checker GP rcOut.csv rc_flex_rules.csv > rc_flex_checked.csv
rc_o_rules_checked = pd.read_csv("rc_o_checked.csv")
rc_flex_rules_checked = pd.read_csv("rc_flex_checked.csv")


!java Checker CN2 fcOut.csv fc_o_rules.csv > fc_o_checked.csv
!java Checker GP fcOut.csv fc_flex_rules.csv > fc_flex_checked.csv
fc_o_rules_checked = pd.read_csv("fc_o_checked.csv")
fc_flex_rules_checked = pd.read_csv("fc_flex_checked.csv")


# Results: Emissions Rules

In [105]:
em_o_rules_checked.sort_values('Precision',ascending=False).head(3).to_csv('results.csv', mode='a', header=True)
em_o_rules_checked.sort_values('Precision',ascending=False).head(3)

,complexity,length,quality,rule,TP,TP%,FP,FP%,Precision
9,-2.0,2.0,0.642857,IF ﻿byMD>=4.0 AND MDs in use>=3.0 THEN emissio...,48,0.029832,13,0.008080,0.786885
2,-2.0,2.0,0.769231,IF ﻿byMD>=5.0 AND EvanDels>=5.0 THEN emissions=1,50,0.031075,16,0.009944,0.757576
5,-5.0,5.0,0.723077,IF ﻿byMD>=5.0 AND MDs in use<=2.0 AND MDs in u...,269,0.167185,132,0.082039,0.670823


In [106]:
em_flex_rules_checked.sort_values('Precision',ascending=False).head(3).to_csv('results.csv', mode='a', header=True)
em_flex_rules_checked.sort_values('Precision',ascending=False).head(3)

,accuracy,f_score,false_neg,false_pos,precison,recall,rule,TP,TP%,FP,High FP%,Precision
19,0.585492,0.0196078431372549,0.990056,0.003110,0.6956521739130435,0.009944,(and EVanDels in [ 2.35 - 6.0 ] (and CycleDel...,16,0.009944,4,0.002486,0.800000
9,0.589637,0.05714285714285714,0.970168,0.010218,0.676056338028169,0.029832,(and EVanDels in [ 2.35 - 6.0 ] CycleDels in ...,48,0.029832,19,0.011809,0.716418
28,0.589378,0.06157489638839551,0.967682,0.012439,0.65,0.032318,(and (or EVanDels in [ 2.35 - 6.0 ] WalkDels ...,52,0.032318,21,0.013052,0.712329


# Results: Time Rules

In [107]:
t_o_rules_checked.sort_values('Precision',ascending=False).head(3).to_csv('results.csv', mode='a', header=True)
t_o_rules_checked.sort_values('Precision',ascending=False).head(3)


,complexity,length,quality,rule,TP,TP%,FP,FP%,Precision
2,-3.0,3.0,0.714286,IF EvanDels>=3.0 AND CycleDels>=2.0 AND EvanDe...,96,0.179439,55,0.102804,0.635762
13,-2.0,2.0,0.333333,IF ﻿byMD>=4.0 AND EvanDels>=4.0 THEN Time=1,96,0.179439,55,0.102804,0.635762
6,-3.0,3.0,0.578947,IF EvanDels>=2.0 AND ﻿byMD<=4.0 AND EvanDels>=...,96,0.179439,55,0.102804,0.635762


In [108]:
t_flex_rules_checked.sort_values('Precision',ascending=False).head(3).to_csv('results.csv', mode='a', header=True)
t_flex_rules_checked.sort_values('Precision',ascending=False).head(3)

,accuracy,f_score,false_neg,false_pos,precison,recall,rule,TP,TP%,FP,High FP%,Precision
37,0.829016,0.16455696202531644,0.878505,0.057143,0.2549019607843137,0.121495,(and (and CycleDels in [ 1.0 - 2.35 ] WalkDel...,65,0.121495,23,0.042991,0.738636
47,0.851295,0.103125,0.938318,0.021654,0.3142857142857143,0.061682,(and (and byMD in [ 1.2 - 4.45 ] MDSinUse in ...,33,0.061682,12,0.022430,0.733333
3,0.826166,0.17466174661746622,0.867290,0.062256,0.25539568345323743,0.132710,(and EVanDels in [ 2.5 - 6.0 ] (and MDSinUse ...,71,0.132710,27,0.050467,0.724490


# Results: Running Cost Rules

In [109]:
rc_o_rules_checked.sort_values('Precision',ascending=False).head(3).to_csv('results.csv', mode='a', header=True)
rc_o_rules_checked.sort_values('Precision',ascending=False).head(3)

,complexity,length,quality,rule,TP,TP%,FP,FP%,Precision
0,-4.0,4.0,0.705882,IF ﻿byMD<=4.0 AND CycleDels<=3.0 AND ﻿byMD<=3....,29,0.828571,24,0.685714,0.54717
1,-1.0,1.0,0.343750,IF CycleDels>=2.0 THEN running cost=1,18,0.514286,18,0.514286,0.50000


In [110]:
rc_flex_rules_checked.sort_values('Precision',ascending=False).head(3).to_csv('results.csv', mode='a', header=True)
rc_flex_rules_checked.sort_values('Precision',ascending=False).head(3)

,accuracy,f_score,false_neg,false_pos,precison,recall,rule,TP,TP%,FP,High FP%,Precision
14,0.900259,0.03508771929824562,0.800000,0.093333,0.019230769230769232,0.200000,(and (not (and CycleDels in [ 1.0 - 2.8 ] (an...,7,0.200000,2,0.057143,0.777778
10,0.781865,0.06236080178173719,0.200000,0.218301,0.03244495944380069,0.800000,(and (or (and CycleDels in [ 1.0 - 2.8 ] EVan...,28,0.800000,9,0.257143,0.756757
16,0.801036,0.06569343065693431,0.228571,0.198693,0.03430749682337993,0.771429,(and (or byMD in [ 1.0 - 3.65 ] CycleDels in ...,27,0.771429,9,0.257143,0.750000


# Results: Fixed Cost Rules

In [111]:
fc_o_rules_checked.sort_values('Precision',ascending=False).head(3).to_csv('results.csv', mode='a', header=True)
fc_o_rules_checked.sort_values('Precision',ascending=False).head(3)

,complexity,length,quality,rule,TP,TP%,FP,FP%,Precision
0,-4.0,4.0,0.812500,IF WalkDels<=2.0 AND ﻿byMD<=4.0 AND CycleDels>...,18,0.108434,10,0.060241,0.642857
3,-2.0,2.0,0.532468,IF ﻿byMD<=3.0 AND EvanDels<=2.0 THEN fixed cos...,166,1.000000,115,0.692771,0.590747
2,-3.0,3.0,0.674157,IF ﻿byMD<=4.0 AND CycleDels>=2.0 AND MDs in us...,166,1.000000,165,0.993976,0.501511


In [112]:
fc_flex_rules_checked.sort_values('Precision',ascending=False).head(3).to_csv('results.csv', mode='a', header=True)
fc_flex_rules_checked.sort_values('Precision',ascending=False).head(3)

,accuracy,f_score,false_neg,false_pos,precison,recall,rule,TP,TP%,FP,High FP%,Precision
2,0.955959,0.16666666666666666,0.897590,0.005685,0.4473684210526316,0.102410,(and byMD in [ 1.0 - 3.5 ] (and CycleDels in ...,17,0.102410,0,0.0,1.0
27,0.952850,0.17272727272727273,0.885542,0.009475,0.35185185185185186,0.114458,(and (and EVanDels in [ 1.0 - 1.05 ] byMD in ...,19,0.114458,0,0.0,1.0
29,0.953886,0.021978021978021976,0.987952,0.003790,0.125,0.012048,(and (and EVanDels in [ 1.0 - 1.05 ] byMD in ...,2,0.012048,0,0.0,1.0
